Import all necessary libraries and install everything you need for training:

In [2]:
# install the libraries necessary for data wrangling, prediction and result analysis
import json
import numpy as np
import pandas as pd
import logging
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix, f1_score,precision_score, recall_score
import torch
from numba import cuda
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyClassifier

In [3]:
# Install transformers
# (this needs to be done on Kaggle each time you start the session)
!pip install -q transformers

# Install the simpletransformers
!pip install -q simpletransformers
from simpletransformers.classification import ClassificationModel

In [4]:
# Install wandb
#!pip install -q wandb

import wandb

# Login to wandb
wandb.login()

In [5]:
# Clean the GPU cache

cuda.select_device(0)
cuda.close()
cuda.select_device(0)
torch.cuda.empty_cache()


### Import the data

In [6]:
# CORE-sub (texts annotated with CORE subcategories)
train_df = pd.read_csv("/kaggle/input/genredatasets/CORE-sub-subset-train.csv", index_col = 0)
# I used a subset of the original dev file during the hyperparameter search because prediction takes very long
dev_df = pd.read_csv("/kaggle/input/genredatasets/CORE-sub-subset-dev-sample.csv", index_col = 0)
test_df = pd.read_csv("/kaggle/input/genredatasets/CORE-sub-subset-test.csv", index_col = 0)

print("CORE-sub train shape: {}, Dev shape: {}, Test shape: {}.".format(train_df.shape, dev_df.shape, test_df.shape))

In [7]:
train_df.head()

## Training and saving

We will use the multilingual XLM-RoBERTa model
https://huggingface.co/xlm-roberta-base

In [8]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [9]:
# Create a list of labels
LABELS = train_df.labels.unique().tolist()
LABELS

In [10]:
print(LABELS)
print(len(LABELS))

In [11]:
# Initialize Wandb
wandb.init(project="CORE-hyperparameter-search", name="training-the-model")

In [12]:
# Calculate how many steps will each epoch have
# Num steps in epoch = training samples / batch size
steps_per_epoch = int(9537/8)
steps_per_epoch

I evaluated per every 5th epoch - per 5960 steps. I first trained the model while evaluating it to find the optimal number of epochs. As previous experiments on CORE-main showed that less than 10 epochs is enough, we will here try up to 15 epochs.

In [ ]:
# Create a TransformerModel and evaluate during training
epoch = 15

roberta_base_model = ClassificationModel(
        "xlmroberta", "xlm-roberta-base",
        num_labels=len(LABELS),
        use_cuda=True,
        args= {
            "overwrite_output_dir": True,
            "num_train_epochs": epoch,
            "train_batch_size":8,
            "learning_rate": 1e-5,
            # Use these parameters if you want to evaluate during training
            "evaluate_during_training": True,
            "evaluate_during_training_steps": steps_per_epoch*5,
            "evaluate_during_training_verbose": True,
            "use_cached_eval_features": True,
            'reprocess_input_data': True,
            "labels_list": LABELS,
            # The following parameters (no_cache, no_save) are commented out if I want to save the model
            "no_cache": True,
            # Disable no_save: True if you want to save the model
            "no_save": True,
            "max_seq_length": 512,
            "save_steps": -1,
            # Only the trained model will be saved - to prevent filling all of the space
            "save_model_every_epoch":False,
            "wandb_project": 'CORE-hyperparameter-search',
            "silent": True,
            }
        )

# Train the model and evaluate during training
roberta_base_model.train_model(train_df, eval_df = dev_df)

Evaluation during training showed that the number of epochs before the eval_loss starts rising is somewhere between epochs 2 and 6. We then trained the model for epochs 2, 4, 6 and 8 to find the optimum number.

In [ ]:
# Create a file to save results into (you can find it under Data: Output). Be careful, run this step only once to not overwrite the results file.
results = []

with open("CORE-main-Experiments-Results.json", "w") as results_file:
    json.dump(results,results_file, indent= "")

In [ ]:
# Open the main results file:

previous_results_file = open("CORE-main-Experiments-Results.json")
previous_results = json.load(previous_results_file)
len(previous_results)

In [13]:
def testing(test_df, test_name, epoch):
    """
    This function takes the test dataset and applies the trained model on it to infer predictions.
    It also prints and saves a confusion matrix, calculates the F1 scores and saves the results in a list of results.

    Args:
    - test_df (pandas DataFrame)
    - test_name
    - epoch: num_train_epochs
    """
    # Get the true labels
    y_true = test_df.labels

    model = roberta_base_model
    
    # Calculate the model's predictions on test
    def make_prediction(input_string):
        return model.predict([input_string])[0][0]

    y_pred = test_df.text.apply(make_prediction)

    # Calculate the scores
    macro = f1_score(y_true, y_pred, labels=LABELS, average="macro")
    micro = f1_score(y_true, y_pred, labels=LABELS,  average="micro")
    print(f"Macro f1: {macro:0.3}, Micro f1: {micro:0.3}")

    # Plot the confusion matrix:
    cm = confusion_matrix(y_true, y_pred, labels=LABELS)
    plt.figure(figsize=(9, 9))
    plt.imshow(cm, cmap="Oranges")
    for (i, j), z in np.ndenumerate(cm):
        plt.text(j, i, '{:d}'.format(z), ha='center', va='center')
    classNames = LABELS
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    tick_marks = np.arange(len(classNames))
    plt.xticks(tick_marks, classNames, rotation=90)
    plt.yticks(tick_marks, classNames)
    plt.title(f"{test_name}")

    plt.tight_layout()
    fig1 = plt.gcf()
    plt.show()
    plt.draw()
    fig1.savefig(f"Confusion-matrix-{test_name}.png",dpi=100)

    # Save the results:
    rezdict = {
        "experiment": test_name,
        "num_train_epochs": epoch,
        "train_batch_size":8,
        "learning_rate": 1e-5,
        "microF1": micro,
        "macroF1": macro,
        "y_true": y_true.to_dict(),
        "y_pred": y_pred.to_dict(),
        }
    previous_results.append(rezdict)

    #Save intermediate results (just in case)
    backup = []
    backup.append(rezdict)
    with open(f"backup-results-{test_name}.json", "w") as backup_file:
        json.dump(backup,backup_file, indent= "")

In [ ]:
# Train the model for various epochs to find the optimum number
epochs = [4, 6, 8]

for epoch in epochs:
    roberta_base_model = ClassificationModel(
                "xlmroberta", "xlm-roberta-base",
                num_labels=len(LABELS),
                use_cuda=True,
                args= {
                    "overwrite_output_dir": True,
                    "num_train_epochs": epoch,
                    "train_batch_size":8,
                    "learning_rate": 1e-5,
                    "labels_list": LABELS,
                    # The following parameters (no_cache, no_save) are commented out if I want to save the model
                    "no_cache": True,
                    # Disable no_save: True if you want to save the model
                    "no_save": True,
                    "max_seq_length": 512,
                    "save_steps": -1,
                    # Only the trained model will be saved - to prevent filling all of the space
                    "save_model_every_epoch":False,
                    "wandb_project": 'CORE-hyperparameter-search',
                    "silent": True,
                    }
                )

    # Train the model
    roberta_base_model.train_model(train_df)
    
    # Test the model on dev_df
    testing(dev_df, f"CORE-sub-dev-epoch-search:{epoch}", epoch)

Results:
- epoch 2: Macro f1: 0.265, Micro f1: 0.641
- epoch 4: Macro f1: 0.382, Micro f1: 0.668
- epoch 6: Macro f1: 0.414, Micro f1: 0.691
- epoch 8: Macro f1: 0.426, Micro f1: 0.682

In [ ]:
# Compare the results by creating a dataframe from the previous_results dictionary:
results_df = pd.DataFrame(previous_results)

results_df

The experiments revealed that the optimum number of epochs is 6.

In [ ]:
# Save the file with updated results.
with open("results/CORE-main-Experiments-Results.json", "w") as results_file:
    json.dump(previous_results,results_file, indent= "")

Train the model and save it.

In [14]:
# Create a TransformerModel
roberta_base_model = ClassificationModel(
        "xlmroberta", "xlm-roberta-base",
        num_labels=len(LABELS),
        use_cuda=True,
        args= {
            "overwrite_output_dir": True,
            "num_train_epochs": 6,
            "train_batch_size":8,
            "learning_rate": 1e-5,
            "labels_list": LABELS,
            # The following parameters are commented out because I want to save the model
            #"no_cache": True,
            # Disable no_save: True if you want to save the model
            #"no_save": True,
            "max_seq_length": 512,
            "save_steps": -1,
            # Only the trained model will be saved - to prevent filling all of the space
            "save_model_every_epoch":False,
            "wandb_project": 'CORE-hyperparameter-search',
            "silent": True,
            }
        )

In [15]:
# Train the model
roberta_base_model.train_model(train_df)

In [16]:
# Save the trained model to Wandb
run = wandb.init(project="CORE-hyperparameter-search", entity="tajak", name="saving-trained-model")
trained_model_artifact = wandb.Artifact("CORE-sub-classifier", type="model", description="A model trained on the CORE dataset with the subcategories as the labels. There are 37 labels: 'News Report/Blog', 'Description of a Thing', 'Description with Intent to Sell', 'Opinion Blog', 'Sports Report', 'Question/Answer Forum', 'Personal Blog', 'Discussion Forum', 'Advice', 'Historical Article', 'Reviews', 'Recipe', 'Information Blog', 'Interview', 'How-to', 'Description of a Person', 'Formal Speech', 'Religious Blogs/Sermons', 'Song Lyrics', 'Short Story', 'FAQ about Information', 'Travel Blog', 'Research Article', 'Encyclopedia Article', 'Persuasive Article or Essay', 'Transcript of Video/Audio', 'Poem', 'Legal terms', 'Course Materials', 'Technical Support', 'Magazine Article', 'Other Information', 'Reader/Viewer Responses', 'Editorial', 'FAQ about How-to', 'Technical Report', 'Letter to Editor'.")
trained_model_artifact.add_dir("/kaggle/working/outputs")
run.log_artifact(trained_model_artifact)